# 📘 Pandas Guide — Data Science Journey


## Table of Contents
1. Introduction
2. Load the Iris Dataset (Main Example)
3. Creating & Exploring DataFrames (Quick Recap)
4. Data Cleaning
5. Indexing, Selection & Filtering
6. Data Transformation
7. Aggregation & Grouping
8. Combining Data
9. Time Series Handling
10. Practice Exercises (Mixed Difficulty)
11. Appendix: Small Demo Datasets (Sales, Dates)


## 1) Introduction
Pandas is a high-level library for **data manipulation**. Two core objects:

- **Series**: 1D labeled array
- **DataFrame**: 2D labeled table with rows & columns

**Import & Settings**

In [43]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 12)
pd.set_option('display.width', 120)


## 2) Load the Iris Dataset (Main Example)
We'll use **Iris** as the running example (classic ML dataset).

In [44]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
cols = ['sepal_length','sepal_width','petal_length','petal_width','class']
df = pd.read_csv(url, names=cols)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [46]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [47]:
df.groupby('class').agg(['mean','min','max'])

sepal_length           sepal_width           petal_length           petal_width          
                        mean  min  max        mean  min  max         mean  min  max        mean  min  max
class                                                                                                    
Iris-setosa            5.006  4.3  5.8       3.418  2.3  4.4        1.464  1.0  1.9       0.244  0.1  0.6
Iris-versicolor        5.936  4.9  7.0       2.770  2.0  3.4        4.260  3.0  5.1       1.326  1.0  1.8
Iris-virginica         6.588  4.9  7.9       2.974  2.2  3.8        5.552  4.5  6.9       2.026  1.4  2.5

## 3) Creating & Exploring DataFrames (Quick Recap)
Create from dicts/lists, inspect, and access metadata.

In [48]:
students = pd.DataFrame({
    'id': [101, 102, 103, 104],
    'name': ['Harry','Ron','Hermione','Luna'],
    'age':  [23, 22, 20, 21],
    'city': ['NY','LA','NY','CHI']
})
students.index.name = 'row_id'
students

,id,name,age,city
row_id,,,,
0,101,Harry,23,NY
1,102,Ron,22,LA
2,103,Hermione,20,NY
3,104,Luna,21,CHI


In [49]:
students.columns, students.index, students.shape

(Index(['id', 'name', 'age', 'city'], dtype='object'),
 RangeIndex(start=0, stop=4, step=1, name='row_id'),
 (4, 4))

In [50]:
students.dtypes

id       int64
name    object
age      int64
city    object
dtype: object

In [51]:
students.head(3)

,id,name,age,city
row_id,,,,
0,101,Harry,23,NY
1,102,Ron,22,LA
2,103,Hermione,20,NY


## 4) Data Cleaning
Tackle missing values, duplicates, renames, dtypes, and string ops.

In [52]:
# Simulate a dirty copy of Iris with NaNs & dupes
dirty = df.copy()
dirty.loc[[0, 5], 'sepal_width'] = np.nan
dirty = pd.concat([dirty, dirty.iloc[[10]]], ignore_index=True)  # add a duplicate row
dirty.head(8)

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,NaN,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,NaN,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa


### 4.1 Missing Values: `.isna()`, `.fillna()`, `.dropna()`

In [53]:
# Find missing
dirty.isna().sum()

sepal_length    0
sepal_width     2
petal_length    0
petal_width     0
class           0
dtype: int64

In [54]:
# Fill missing with median per column (numeric only)
filled = dirty.copy()
for c in ['sepal_length','sepal_width','petal_length','petal_width']:
    filled[c] = filled[c].fillna(filled[c].median())
filled.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
class           0
dtype: int64

In [55]:
# Alternatively, drop rows with any NaN (only if permissible)
dropped = dirty.dropna()
dropped.shape

(149, 5)

### 4.2 Remove Duplicates: `.drop_duplicates()`

In [56]:
no_dupes = filled.drop_duplicates()
no_dupes.shape

(147, 5)

### 4.3 Rename Columns: `.rename()`

In [57]:
renamed = no_dupes.rename(columns={'sepal_length':'sepal_len','sepal_width':'sepal_wid'})
renamed.head(3)

,sepal_len,sepal_wid,petal_length,petal_width,class
0,5.1,3.0,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


### 4.4 Change Dtypes: `.astype()`

In [58]:
# Cast select features to float32 for memory efficiency
typed = renamed.copy()
for c in ['sepal_len','sepal_wid','petal_length','petal_width']:
    typed[c] = typed[c].astype('float32')
typed.dtypes

sepal_len       float32
sepal_wid       float32
petal_length    float32
petal_width     float32
class            object
dtype: object

### 4.5 String Operations: `.str.lower()`, `.str.contains()`

In [59]:
# Lowercase class & search
str_demo = typed.copy()
str_demo['class'] = str_demo['class'].str.lower()
mask_versi = str_demo['class'].str.contains('versicolor')
str_demo[mask_versi].head(3)

,sepal_len,sepal_wid,petal_length,petal_width,class
50,7.0,3.2,4.7,1.4,iris-versicolor
51,6.4,3.2,4.5,1.5,iris-versicolor
52,6.9,3.1,4.9,1.5,iris-versicolor


## 5) Indexing, Selection & Filtering
Access columns/rows, boolean masks, and multiple conditions.

In [60]:
df['sepal_length'].head()

0    5.1
1    4.9
2    4.7
3    4.6
4    5.0
Name: sepal_length, dtype: float64

In [61]:
df[['sepal_length','sepal_width']].head()

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


In [62]:
df.loc[0]  # label-based single row

sepal_length            5.1
sepal_width             3.5
petal_length            1.4
petal_width             0.2
class           Iris-setosa
Name: 0, dtype: object

In [63]:
df.iloc[0]  # position-based single row

sepal_length            5.1
sepal_width             3.5
petal_length            1.4
petal_width             0.2
class           Iris-setosa
Name: 0, dtype: object

In [64]:
df.loc[0:4, ['sepal_length','class']]  # label slice + column subset

,sepal_length,class
0,5.1,Iris-setosa
1,4.9,Iris-setosa
2,4.7,Iris-setosa
3,4.6,Iris-setosa
4,5.0,Iris-setosa


In [65]:
df[df['sepal_length'] > 6.0].head()

,sepal_length,sepal_width,petal_length,petal_width,class
50,7.0,3.2,4.7,1.4,Iris-versicolor
51,6.4,3.2,4.5,1.5,Iris-versicolor
52,6.9,3.1,4.9,1.5,Iris-versicolor
54,6.5,2.8,4.6,1.5,Iris-versicolor
56,6.3,3.3,4.7,1.6,Iris-versicolor


In [66]:
df[(df['sepal_length'] > 6.0) & (df['sepal_width'] >= 3.0)].head()

,sepal_length,sepal_width,petal_length,petal_width,class
50,7.0,3.2,4.7,1.4,Iris-versicolor
51,6.4,3.2,4.5,1.5,Iris-versicolor
52,6.9,3.1,4.9,1.5,Iris-versicolor
56,6.3,3.3,4.7,1.6,Iris-versicolor
65,6.7,3.1,4.4,1.4,Iris-versicolor


## 6) Data Transformation
Sort, add/modify columns, apply functions, and bin data.

### 6.1 Sorting: `.sort_values()`, `.sort_index()`

In [67]:
df.sort_values(by=['sepal_length','sepal_width']).head(6)

,sepal_length,sepal_width,petal_length,petal_width,class
13,4.3,3.0,1.1,0.1,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
38,4.4,3.0,1.3,0.2,Iris-setosa
42,4.4,3.2,1.3,0.2,Iris-setosa
41,4.5,2.3,1.3,0.3,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa


In [68]:
df.sort_index().head(3)

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


### 6.2 Adding & Modifying Columns

In [69]:
trans = df.copy()
trans['sepal_ratio'] = trans['sepal_length'] / trans['sepal_width']
trans.head(3)

,sepal_length,sepal_width,petal_length,petal_width,class,sepal_ratio
0,5.1,3.5,1.4,0.2,Iris-setosa,1.457143
1,4.9,3.0,1.4,0.2,Iris-setosa,1.633333
2,4.7,3.2,1.3,0.2,Iris-setosa,1.468750


### 6.3 Applying Functions: `.apply()`, `.map()`, `.applymap()`

In [70]:
# Column-wise apply
trans['len_bucket'] = trans['sepal_length'].apply(lambda x: 'long' if x >= 6.0 else 'short')
trans['len_bucket'].value_counts()

len_bucket
short    83
long     67
Name: count, dtype: int64

In [71]:
# Map on a categorical/label column
class_map = {'Iris-setosa':'setosa','Iris-versicolor':'versicolor','Iris-virginica':'virginica'}
mapped = df.copy()
mapped['class_slim'] = mapped['class'].map(class_map)
mapped[['class','class_slim']].head(3)

,class,class_slim
0,Iris-setosa,setosa
1,Iris-setosa,setosa
2,Iris-setosa,setosa


In [72]:
# applymap on a small numeric frame (element-wise)
small = pd.DataFrame({'a':[1,2,3],'b':[4,5,6]})
small.applymap(lambda v: v*10)

/var/folders/jd/xrdcg2j963gfc26_77vyn7rr0000gn/T/ipykernel_4357/3318115260.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  small.applymap(lambda v: v*10)


,a,b
0,10,40
1,20,50
2,30,60


### 6.4 Binning & Categorizing Data

In [73]:
bins = [0, 2.5, 5.0, 7.5]
labels = ['very small','small','large']
binned = df.copy()
binned['sepal_len_bin'] = pd.cut(binned['sepal_length'], bins=bins, labels=labels, include_lowest=True)
binned[['sepal_length','sepal_len_bin']].head(6)

,sepal_length,sepal_len_bin
0,5.1,large
1,4.9,small
2,4.7,small
3,4.6,small
4,5.0,small
5,5.4,large


## 7) Aggregation & Grouping
Group data and compute aggregations. Use `.agg()` for flexibility.

In [74]:
df.groupby('class')['sepal_length'].agg(['count','mean','std','min','max'])

,count,mean,std,min,max
class,,,,,
Iris-setosa,50,5.006,0.352490,4.3,5.8
Iris-versicolor,50,5.936,0.516171,4.9,7.0
Iris-virginica,50,6.588,0.635880,4.9,7.9


In [75]:
df.groupby('class').agg(
    sepal_length_mean=('sepal_length','mean'),
    sepal_width_median=('sepal_width','median'),
    petal_length_p90=('petal_length', lambda s: s.quantile(0.90))
)

,sepal_length_mean,sepal_width_median,petal_length_p90
class,,,
Iris-setosa,5.006,3.4,1.70
Iris-versicolor,5.936,2.8,4.80
Iris-virginica,6.588,3.0,6.31


### Pivot Tables: `.pivot_table()` and Crosstabs: `pd.crosstab()`

In [76]:
pd.pivot_table(df, values='sepal_length', index='class', aggfunc=['mean','median'])

,mean,median
,sepal_length,sepal_length
class,,
Iris-setosa,5.006,5.0
Iris-versicolor,5.936,5.9
Iris-virginica,6.588,6.5


In [77]:
pd.crosstab(binned['sepal_len_bin'], df['class'])

class,Iris-setosa,Iris-versicolor,Iris-virginica
sepal_len_bin,,,
small,28,3,1
large,22,47,43


## 8) Combining Data
Concatenate, merge, and join. Prefer `pd.concat` over deprecated `.append()`.

In [78]:
# Example: vertical concat
part1 = df.iloc[:3]
part2 = df.iloc[3:6]
pd.concat([part1, part2], ignore_index=True)

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa


In [79]:
# Example: merge using a small Sales dataset
sales = pd.DataFrame({
    'order_id':[1,2,3,4],
    'customer_id':[101,102,101,103],
    'amount':[250, 120, 300, 180]
})
customers = pd.DataFrame({
    'customer_id':[101,102,104],
    'name':['Alice','Bob','Charlie']
})
pd.merge(sales, customers, on='customer_id', how='left')

,order_id,customer_id,amount,name
0,1,101,250,Alice
1,2,102,120,Bob
2,3,101,300,Alice
3,4,103,180,NaN


> ℹ️ **Note**: `.append()` is deprecated. Use `pd.concat([df1, df2], ignore_index=True)`.

## 9) Time Series Handling
Convert to datetime, filter by dates, and resample.

In [80]:
ts = pd.DataFrame({
    'date': pd.date_range('2025-01-01', periods=8, freq='D'),
    'value': [10,12,9,11,13,15,14,16]
})
ts.head()

,date,value
0,2025-01-01,10
1,2025-01-02,12
2,2025-01-03,9
3,2025-01-04,11
4,2025-01-05,13


In [81]:
ts.dtypes

date     datetime64[ns]
value             int64
dtype: object

In [82]:
# Ensure datetime dtype then set index
ts['date'] = pd.to_datetime(ts['date'])
ts = ts.set_index('date')
ts

,value
date,
2025-01-01,10
2025-01-02,12
2025-01-03,9
2025-01-04,11
2025-01-05,13
2025-01-06,15
2025-01-07,14
2025-01-08,16


In [83]:
# Date-based filtering
ts['2025-01-03':'2025-01-06']

,value
date,
2025-01-03,9
2025-01-04,11
2025-01-05,13
2025-01-06,15


In [84]:
# Resample to 2-day frequency (sum)
ts.resample('2D').sum()

,value
date,
2025-01-01,22
2025-01-03,20
2025-01-05,28
2025-01-07,30


## 10) Practice Exercises — Mixed Difficulty
Tackle these with Iris (`df`), the binned frame (`binned`), the sales tables, and `ts`.

**Easy**
1. Select all rows where `sepal_width > 3.2`.
2. Create a new column `petal_area = petal_length * petal_width`.
3. Remove duplicate rows (if any) from a copy of `df`.

**Intermediate**
4. Compute **mean** `sepal_length` *per class* and sort descending.
5. Using `sales` and `customers`, compute total `amount` **per customer name**.
6. Using `ts`, calculate a 3-day rolling **mean** on `value`.

**Challenging**
7. Build a tidy summary using `.groupby(...).agg(...)` showing for each `class`: mean/median of all four numeric columns.
8. Create a column that bins `petal_length` into **quartiles** using `pd.qcut`, then produce a crosstab vs `class`.
9. For `sales`, create a daily series of synthetic order dates, set as index, and **resample monthly** to total revenue.

_Hints_: prefer vectorized ops, use `merge`, `groupby`, `agg`, `cut`/`qcut`, and `resample`. Avoid deprecated `.append()`.

## 11) Appendix — Small Demo Datasets
### Sales Data
```
sales = pd.DataFrame({'order_id':[1,2,3,4], 'customer_id':[101,102,101,103], 'amount':[250,120,300,180]})
customers = pd.DataFrame({'customer_id':[101,102,104], 'name':['Alice','Bob','Charlie']})
```
### Datetime Series
```
ts = pd.DataFrame({'date': pd.date_range('2025-01-01', periods=8, freq='D'), 'value':[10,12,9,11,13,15,14,16]})
```

---
### Best Practices & Notes
- Prefer `pd.concat` over `.append()` (deprecated).
- Use `.loc`/`.iloc` for label/position selection respectively; avoid chained indexing.
- Use `.agg()` for clean multi-aggregation.
- For memory, consider `astype('float32')` on numeric columns when precision allows.
- For strings: normalize with `.str.lower()` and filter with `.str.contains()`.